In [25]:
import polars as pl
import pandas as pd
import numpy as np
import preprocessing.transform_data as td
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier
import preprocessing.modeling as md
from sktime.transformations.panel.rocket import Rocket, MiniRocketMultivariate
from sklearn.ensemble import RandomForestClassifier

In [26]:
df_raw = pl.read_parquet("./data/2022_ST4000.parquet")

In [29]:
df_raw

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,faildate,Time_till_failure,failing_in14days
date,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,date,duration[ms],i32
2022-03-11,"""Z305DEMX""","""ST4000DM000""",4000787030016,0,114,61208648,92,0,100,16,100,0,84,274860655,38,54806,100,0,100,15,100,0,100,0,100,1,83,34459,19,19,100,0,100,0,200,0,null,null,0
2022-03-11,"""S301NGZV""","""ST4000DM000""",4000787030016,0,117,145639880,91,0,100,19,100,0,84,302259307,37,55790,100,0,100,19,100,0,100,0,100,0,92,17578,27,27,100,0,100,0,200,0,null,null,0
2022-03-11,"""S3010M7R""","""ST4000DM000""",4000787030016,0,117,140768432,91,0,100,19,100,0,87,610381474,32,59756,100,0,100,19,100,0,100,0,100,0,82,37679,24,24,100,0,100,0,200,0,null,null,0
2022-03-11,"""S3010M7H""","""ST4000DM000""",4000787030016,0,117,122568920,91,0,100,26,100,0,84,269858269,33,59457,100,0,100,26,99,1,100,0,100,0,17,166861,24,24,100,0,100,0,200,0,null,null,0
2022-03-11,"""S3010M7E""","""ST4000DM000""",4000787030016,0,119,215212568,92,0,100,19,100,0,88,755694637,32,59761,100,0,100,19,100,0,100,0,100,0,81,39310,23,23,100,0,100,0,200,0,null,null,0
2022-03-11,"""S300YQM3""","""ST4000DM000""",4000787030016,0,118,180818272,92,0,100,9,100,0,87,493784324,32,59766,100,0,100,9,100,0,100,0,100,0,80,40671,21,21,100,0,100,0,200,0,null,null,0
2022-03-11,"""Z305NZM9""","""ST4000DM000""",4000787030016,0,117,146451696,91,0,100,10,100,0,90,982238762,41,52489,100,0,100,10,100,0,100,0,100,1,98,4376,27,27,100,0,100,0,200,0,null,null,0
2022-03-11,"""Z305AJF7""","""ST4000DM000""",4000787030016,0,111,32456440,91,0,100,8,100,0,82,193449288,39,53947,100,0,100,8,100,0,100,0,100,0,96,8840,28,28,100,0,100,0,200,0,null,null,0
2022-03-11,"""S301NGZ7""","""ST4000DM000""",4000787030016,0,117,167698376,92,0,100,24,100,0,87,483475289,36,56293,100,0,100,24,100,0,100,0,100,0,96,8960,28,28,100,0,100,0,200,0,null,null,0


In [3]:
#failing_drives = df_raw.filter(pl.col("failure") == 1)["serial_number"]

In [4]:
#df_raw = df_raw.filter(pl.col("serial_number").is_in(failing_drives))

In [27]:
df_raw = td.filter_out_inconsistent_drives(df_raw)

In [28]:
Train, Test = td.custom_train_test_split(df_raw)

In [30]:
td.create_chunks(df_raw,Train, name="Train_30")

0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500


KeyboardInterrupt: 

In [31]:
td.create_chunks(df_raw,Test, name="Test_30")

0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 1500
100 of 1500
200 of 1500
300 of 1500
400 of 1500
500 of 1500
600 of 1500
700 of 1500
800 of 1500
900 of 1500
1000 of 1500
1100 of 1500
1200 of 1500
1300 of 1500
1400 of 1500
0 of 234
100 of 234
200 of 234


In [68]:
X_train = pd.read_parquet(f"data/datachunks/Failed_Train_30_chunk1.parquet")
X_train2 = pd.read_parquet(f"data/datachunks/Train_30_chunk1.parquet")
X_train3 = pd.read_parquet(f"data/datachunks/Train_30_chunk2.parquet")
X_train4 = pd.read_parquet(f"data/datachunks/Train_30_chunk3.parquet")
X_test = pd.read_parquet(f"data/datachunks/Failed_Test_30_chunk1.parquet")
X_test2 = pd.read_parquet(f"data/datachunks/Test_30_chunk1.parquet")
X_test3 = pd.read_parquet(f"data/datachunks/Test_30_chunk2.parquet")
X_test4 = pd.read_parquet(f"data/datachunks/Test_30_chunk3.parquet")

In [69]:
X_train = pd.concat([X_train, X_train2, X_train3,X_train4])


In [70]:
X_test = pd.concat([X_test, X_test2, X_test3, X_test4])

In [71]:
a1 = X_test.index.get_level_values(0).to_series()
a2 = a1.ne(a1.shift()).cumsum() - 1

mux = pd.MultiIndex.from_arrays([a2, X_test.index.get_level_values(1)], names=X_test.index.names)

X_test.index = mux

In [72]:
a1 = X_train.index.get_level_values(0).to_series()
a2 = a1.ne(a1.shift()).cumsum() - 1

mux = pd.MultiIndex.from_arrays([a2, X_train.index.get_level_values(1)], names=X_train.index.names)

X_train.index = mux

In [73]:
y_train = X_train.iloc[29::30, :]["failing_in14days"].astype(int).reset_index(drop=True).to_list()
y_test = X_test.iloc[29::30, :]["failing_in14days"].astype(int).reset_index(drop=True).to_list()

In [83]:
y_train.count(1)

581

In [75]:
from sktime.transformations.panel.rocket import Rocket
from sktime.datatypes import convert,convert_to,mtype,check_raise

In [42]:
rocket = Rocket()

In [76]:
col_list = td.feature_list(target=False)
X_train = X_train[col_list]
X_test = X_test[col_list]

In [9]:
rocket.fit(X_train)

Rocket()

In [80]:
X_train

smart_1_raw  smart_3_raw  smart_4_raw  smart_5_raw  \
instances timepoints                                                       
0         0              70618024            0           18            0   
          1              59949440            0           18            0   
          2             137973224            0           18            0   
          3             232769008            0           18            0   
          4              75459192            0           18            0   
...                           ...          ...          ...          ...   
55200     25             63136696            0           17            0   
          26            127583608            0           17            0   
          27             38231808            0           17            0   
          28            220590552            0           17            0   
          29             16046448            0           17            0   

                      smart_7_raw  smart_9_raw  smart_10_raw  smart_12_raw  \
instances timepoints                                                         
0         0             630593177        55846             0            18   
          1             636206328        55870             0            18   
          2             641574730        55894             0            18   
          3             647056236        55918             0            18   
          4             652437848        55941             0            18   
...                           ...          ...           ...           ...   
55200     25            903068582        63289             0            17   
          26            906743934        63313             0            17   
          27            909790929        63337             0            17   
          28            912054371        63361             0            17   
          29            914188563        63385             0            17   

                      smart_187_raw  smart_188_raw  smart_192_raw  \
instances timepoints                                                
0         0                      51              0              0   
          1                      54              0              0   
          2                      54              0              0   
          3                      58              0              0   
          4                      71              0              0   
...                             ...            ...            ...   
55200     25                      0              0              2   
          26                      0              0              2   
          27                      0              0              2   
          28                      0              0              2   
          29                      0              0              2   

                      smart_193_raw  smart_194_raw  smart_197_raw  \
instances timepoints                                                
0         0                   19301             33             32   
          1                   19301             33             32   
          2                   19301             33             32   
          3                   19301             33             40   
          4                   19301             32             56   
...                             ...            ...            ...   
55200     25                   8873             25              0   
          26                   8873             25              0   
          27                   8873             24              0   
          28                   8873             25              0   
          29                   8873             25              0   

                      smart_198_raw  smart_199_raw  
instances timepoints                                
0         0                      32              0  
          1                      32              0  
          

In [81]:
X_test

smart_1_raw  smart_3_raw  smart_4_raw  smart_5_raw  \
instances timepoints                                                       
0         0              83098424            0           19         1248   
          1             211891096            0           21         1248   
          2             149493232            0           21         1248   
          3              44264128            0           21         1248   
          4             230012616            0           21         1248   
...                           ...          ...          ...          ...   
53176     25            236064304            0            9            0   
          26             25994056            0            9            0   
          27            179871160            0            9            0   
          28             45339136            0            9            0   
          29              7213168            0            9            0   

                      smart_7_raw  smart_9_raw  smart_10_raw  smart_12_raw  \
instances timepoints                                                         
0         0             851229748        54136             0            19   
          1             852122684        54162             0            21   
          2             853731691        54186             0            21   
          3             854556219        54210             0            21   
          4             855324185        54233             0            21   
...                           ...          ...           ...           ...   
53176     25            441622009        59983             0             9   
          26            442960904        60007             0             9   
          27            444881143        60031             0             9   
          28            445647109        60043             0             9   
          29            448130180        60066             0             9   

                      smart_187_raw  smart_188_raw  smart_192_raw  \
instances timepoints                                                
0         0                     105              0              2   
          1                     105              0              4   
          2                     105              0              4   
          3                     105              0              4   
          4                     105              0              4   
...                             ...            ...            ...   
53176     25                      0              0              0   
          26                      0              0              0   
          27                      0              0              0   
          28                      0              0              0   
          29                      0              0              0   

                      smart_193_raw  smart_194_raw  smart_197_raw  \
instances timepoints                                                
0         0                    7335             21              0   
          1                    7337             21              0   
          2                    7337             21              0   
          3                    7337             21              0   
          4                    7337             21              0   
...                             ...            ...            ...   
53176     25                  50128             28              0   
          26                  50128             26              0   
          27                  50128             28              0   
          28                  50128             26              0   
          29                  50128             27              0   

                      smart_198_raw  smart_199_raw  
instances timepoints                                
0         0                       0              0  
          1                       0              0  
          

In [77]:
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)
X_train_transformed_mini = minirocket_multi.transform(X_train)
X_test_transformed_mini = minirocket_multi.transform(X_test)

In [10]:
X_train_transformed = rocket.transform(X_train)
X_test_transformed = rocket.transform(X_test)

In [156]:
# scaler = StandardScaler(with_mean=False)
# X_train_scaled = scaler.fit_transform(X_train_transformed)


In [165]:

#ros = RandomOverSampler(random_state=150)
#X_train_ros, y_train_ros = ros.fit_resample(X_train_transformed, y_train)

In [76]:
#smote = SMOTE()
#X_train_smote, y_train_smote = smote.fit_resample(X_train_transformed, y_train)

In [37]:
# param_grid = {"loss" : ["log_loss"], 
#               "penalty" : ["l2"],
#               "alpha" : [0.001, 0.01, 0.0001],
#              }

# # Instantiate gridsearch and define the metric to optimize 
# gs = GridSearchCV(SGDClassifier(max_iter=1000), param_grid, scoring='f1',
#                   cv=4, verbose=3, n_jobs=-1)

# gs.fit(X_train_transformed, y_train)

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV 2/4] END alpha=0.01, loss=log_loss, penalty=l2;, score=0.338 total time= 1.3min
[CV 2/4] END alpha=0.001, loss=log_loss, penalty=l2;, score=0.379 total time= 2.7min
[CV 3/4] END alpha=0.01, loss=log_loss, penalty=l2;, score=0.296 total time= 3.8min
[CV 1/4] END alpha=0.0001, loss=log_loss, penalty=l2;, score=0.303 total time= 2.6min
[CV 1/4] END alpha=0.001, loss=log_loss, penalty=l2;, score=0.297 total time= 4.0min
[CV 4/4] END alpha=0.001, loss=log_loss, penalty=l2;, score=0.303 total time= 4.0min
[CV 3/4] END alpha=0.0001, loss=log_loss, penalty=l2;, score=0.325 total time=   9.7s
[CV 2/4] END alpha=0.0001, loss=log_loss, penalty=l2;, score=0.364 total time= 1.2min
[CV 3/4] END alpha=0.001, loss=log_loss, penalty=l2;, score=0.337 total time= 4.0min
[CV 1/4] END alpha=0.01, loss=log_loss, penalty=l2;, score=0.290 total time= 4.0min
[CV 4/4] END alpha=0.0001, loss=log_loss, penalty=l2;, score=0.332 total time=   8.0s
[CV 

GridSearchCV(cv=4, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.0001], 'loss': ['log_loss'],
                         'penalty': ['l2']},
             scoring='f1', verbose=3)

In [17]:
sgd = SGDClassifier(loss="log_loss", n_jobs=-1, max_iter=1000, alpha=0.001)
md.fit_predict_evaluate(sgd, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9053956834532374
recall:  0.42152466367713004
F1:  0.5884194053208138
[[2329    5]
 [ 258  188]]
Performance on test data: 
accuracy:  0.8720430107526882
recall:  0.3221476510067114
F1:  0.44651162790697674
[[763  18]
 [101  48]]


In [78]:
sgd1 = SGDClassifier(loss="log_loss", n_jobs=-1, max_iter=1000, alpha=0.0001)
md.fit_predict_evaluate(sgd1, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9927718700748175
recall:  0.4664371772805508
F1:  0.5759829968119022
[[54531    89]
 [  310   271]]
Performance on test data: 
accuracy:  0.9953363296161875
recall:  0.4
F1:  0.48333333333333334
[[52813    74]
 [  174   116]]


In [65]:
sgd2 = SGDClassifier(loss="log_loss", n_jobs=-1, max_iter=1000, alpha=0.0001)
md.fit_predict_evaluate(sgd2, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9903159231624067
recall:  0.4085820895522388
F1:  0.5447761194029851
[[37209    49]
 [  317   219]]
Performance on test data: 
accuracy:  0.9941883207599889
recall:  0.3333333333333333
F1:  0.4316939890710383
[[35503    50]
 [  158    79]]


In [19]:
dtree = DecisionTreeClassifier(max_depth=30)
md.fit_predict_evaluate(dtree, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9949640287769784
recall:  0.968609865470852
F1:  0.9840546697038723
[[2334    0]
 [  14  432]]
Performance on test data: 
accuracy:  0.8064516129032258
recall:  0.46308724832214765
F1:  0.4339622641509434
[[681 100]
 [ 80  69]]


In [48]:
dtree1 = DecisionTreeClassifier(max_depth=30)
md.fit_predict_evaluate(dtree1, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9955968930886063
recall:  0.8187372708757638
F1:  0.9003359462486002
[[19722     0]
 [   89   402]]
Performance on test data: 
accuracy:  0.9877664201826881
recall:  0.4708994708994709
F1:  0.44168734491315137
[[18078   125]
 [  100    89]]


In [66]:
dtree2 = DecisionTreeClassifier(max_depth=30)
md.fit_predict_evaluate(dtree2, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9971953220087845
recall:  0.8022388059701493
F1:  0.8902691511387164
[[37258     0]
 [  106   430]]
Performance on test data: 
accuracy:  0.9925118748253702
recall:  0.48945147679324896
F1:  0.46399999999999997
[[35406   147]
 [  121   116]]


In [24]:
rforest= RandomForestClassifier(max_depth=20)
md.fit_predict_evaluate(rforest, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9859712230215827
recall:  0.9125560538116592
F1:  0.9542790152403283
[[2334    0]
 [  39  407]]
Performance on test data: 
accuracy:  0.8731182795698925
recall:  0.2550335570469799
F1:  0.3917525773195877
[[774   7]
 [111  38]]


In [50]:
rforest1= RandomForestClassifier(max_depth=40)
md.fit_predict_evaluate(rforest1, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9999505268886361
recall:  0.9979633401221996
F1:  0.9989806320081549
[[19722     0]
 [    1   490]]
Performance on test data: 
accuracy:  0.9936929099608526
recall:  0.4444444444444444
F1:  0.5915492957746479
[[18192    11]
 [  105    84]]


In [52]:
rforest2= RandomForestClassifier(max_depth=25)
md.fit_predict_evaluate(rforest2, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9987136991045367
recall:  0.9470468431771895
F1:  0.9728033472803348
[[19722     0]
 [   26   465]]
Performance on test data: 
accuracy:  0.9935841670291431
recall:  0.43386243386243384
F1:  0.5815602836879432
[[18192    11]
 [  107    82]]


In [67]:
rforest3= RandomForestClassifier(max_depth=40)
md.fit_predict_evaluate(rforest3, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9999735407736677
recall:  0.9981343283582089
F1:  0.9990662931839402
[[37258     0]
 [    1   535]]
Performance on test data: 
accuracy:  0.996311818943839
recall:  0.5189873417721519
F1:  0.6507936507936507
[[35535    18]
 [  114   123]]


In [79]:
rforest4= RandomForestClassifier(max_depth=40)
md.fit_predict_evaluate(rforest4, X_train_transformed_mini, y_train, X_test_transformed_mini, y_test)

Performance on train data: 
accuracy:  0.9999094219307622
recall:  0.9913941480206541
F1:  0.9956784788245463
[[54620     0]
 [    5   576]]
Performance on test data: 
accuracy:  0.9976681648080937
recall:  0.6310344827586207
F1:  0.746938775510204
[[52870    17]
 [  107   183]]


In [141]:
y_test =np.array(y_test)

In [142]:
diff = y_test-y_pred
print('diff: ',diff)

# Correct is 0 
# FP is -1 
# FN is 1
print('Correctly classified: ', np.where(diff == 0)[0])
print('Incorrectly classified: ', np.where(diff != 0)[0])
print('failing in 14 days: ', np.where(y_test == 1)[0])
print('False positives: ', np.where(diff == -1)[0])
print('False negatives: ', np.where(diff == 1)[0])

diff:  [0 0 0 ... 0 0 1]
Correctly classified:  [   0    1    2 ... 2010 2012 2013]
Incorrectly classified:  [  21   33   46   71   77   92   99  136  145  155  158  166  178  197
  204  209  231  237  246  252  253  254  256  278  283  290  319  337
  339  344  351  365  382  390  407  416  431  456  459  460  499  511
  516  530  535  539  540  547  551  559  574  587  592  614  616  629
  654  667  678  682  685  686  688  706  710  715  725  736  740  798
  819  837  843  846  848  856  863  874  875  895  917  937  941  942
  943  960  976  985  991 1011 1015 1037 1040 1043 1058 1059 1060 1061
 1065 1069 1073 1076 1085 1088 1108 1115 1120 1121 1128 1141 1143 1144
 1150 1167 1170 1171 1172 1173 1183 1204 1205 1208 1221 1226 1236 1244
 1253 1255 1272 1275 1276 1289 1291 1297 1298 1314 1315 1322 1331 1332
 1348 1360 1365 1412 1413 1434 1441 1455 1456 1475 1477 1482 1484 1494
 1512 1517 1525 1575 1588 1590 1592 1593 1599 1603 1631 1662 1678 1683
 1723 1725 1733 1734 1739 1743 1766 177